In [ ]:
import pandas as pd
import numpy as np
import statsmodels.stats.multitest as ssm
import scipy.stats as stats
import sklearn 

### Problem 1.4

In [59]:
golub_cl = pd.read_csv("/Users/stephenandrews/Downloads/MIT Schoolwork/Spring 2025/6.3730/6373Pset1/data_and_materials/golub_data/golub_cl.csv")
golub_gnames = pd.read_csv("/Users/stephenandrews/Downloads/MIT Schoolwork/Spring 2025/6.3730/6373Pset1/data_and_materials/golub_data/golub_gnames.csv")
golub = pd.read_csv("/Users/stephenandrews/Downloads/MIT Schoolwork/Spring 2025/6.3730/6373Pset1/data_and_materials/golub_data/golub.csv")


# print(golub_cl.head())
# print('------------')
# print(golub_gnames.head())
# print('------------')
# print(golub.head())

golub_cl_np = golub_cl.to_numpy()
golub_gnames_np = golub_gnames.to_numpy()
golub_np = golub.to_numpy()[:, 1:]


In [36]:
#Getting Means for ALL vs AML for each gene

AML_mask = (golub_cl_np[:, 1]).astype(bool) # True for patients with AML, False otherwise

AML_array = golub_np[:, AML_mask]
mu_M = np.mean(AML_array, axis=1)

ALL_array = golub_np[:, ~AML_mask]
mu_L = np.mean(ALL_array, axis=1)


In [58]:
# Calculate p-values 
t_stats, p_values = stats.ttest_ind(ALL_array, AML_array, axis = 1, equal_var=False)

parti = np.where(p_values < 0.05) # uncorrected p-values

partii, _, _, _ = ssm.multipletests(p_values, method='holm') # holm-bonferroni
partii = np.where(partii)


partiii, _, _, _ = ssm.multipletests(p_values, method='fdr_bh') # benjamini-hochberg
partiii = np.where(partiii)


print("# of genes associated with diff tumor types with uncorrected p-values: ", len(parti[0]))
print("# of genes associated with diff tumor types using Holm-Bonferroni Correction: ", len(partii[0]))
print("# of genes associated with diff tumor types using Benjamini-Hochberg Correction: ", len(partiii[0]))


# of genes associated with diff tumor types with uncorrected p-values:  1078
# of genes associated with diff tumor types using Holm-Bonferroni Correction:  103
# of genes associated with diff tumor types using Benjamini-Hochberg Correction:  695


### Problem 1.5a

In [ ]:
X = .tonumpy()